# Wear evaluation of WC-Cu matrix composite based on SEM images
# WC-Cu基胎体磨损电镜图像识别

> 来源/Source：[Colab Notebook](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5) in [Detectron2 project](https://github.com/facebookresearch/detectron2)

> 参考/Reference1：[dyh/unbox_detecting_tunnel_fissure](https://github.com/dyh/unbox_detecting_tunnel_fissure)

> 参考/Reference2：[TannerGilbert/Detectron2-Train-a-Instance-Segmentation-Model](https://github.com/TannerGilbert/Detectron2-Train-a-Instance-Segmentation-Model)

## 0. Google Drive (storing data) + Google Colab (computing power)
## 0. Google Drive (数据储存) + Google Colab (算力平台)

### 0.1 Check out GPU info 
### 0.1 检查GPU信息
> if not GPU, click "Runtime"→"change runtime type"→"Hardware accelerator"→"GPU"→"SAVE"

> 不是GPU的话请点击“代码执行程序”→“更改运行时类型”→“硬件加速器”→“GPU”→保存

In [ ]:
!nvidia-smi

### 0.2 mount google drive folder
### 0.2 装载GoogleDrive云端网盘
> 在打开链接弹出的窗口中复制验证码后出现的输入框后回车

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 0.3 Import labelled images
### 0.3 导入标注过的图像数据集文件

In [ ]:
!git clone 'https://github.com/sunwucheng/IDB_Wear.git' '/content/drive/MyDrive/IDB_Wear'
!wget -P '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108' https://github.com/sunwucheng/IDB_Wear/releases/download/v0.1/WC-Cu_20210108_output.zip
!unzip '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/WC-Cu_20210108_output.zip' -d '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108'

## 1. Install Mask R-CNN framework
## 1. 安装Mask R-CNN算法框架

### 1.1 Install dependencies
### 1.1 安装依赖项

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

### 1.2 Install detectron2
### 1.2 安装detectron2
> click **[ RESTART RUNTIME ]** button at the end after all text printed

> 输出完文本后点击其末尾的**[ RESTART RUNTIME ]**按钮重启

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

---

***please make sure of that you have click the [ RESTART RUNTIME ] button -> [ YES ] button to restart colab runtime***

***请确认您点击了 [ RESTART RUNTIME ] 按钮 -> [ 是 ] 按钮，来重新加载 colab 运行时***


---

### 1.3 Import modules
### 1.3 导入模块

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import GenericMask
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

## 2. Register the datasets
## 2. 注册数据集

### 2.1A Register with this one if you label with VGG IMage Annotator (VIA)
### 2.1A 注册图像数据集(VIA标注的用这个)

In [ ]:
def get_wear_dicts(img_dir):
  json_file = os.path.join(img_dir, "via_region_data.json")
  with open(json_file) as f:
      imgs_anns = json.load(f)

  dataset_dicts = []
  for idx, v in enumerate(imgs_anns.values()):
      record = {}
      
      filename = os.path.join(img_dir, v["filename"])
      height, width = cv2.imread(filename).shape[:2]
      
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      list_annos = v["regions"]

      objs = []
      # for _, anno in annos.items():
      for dict_anno in list_annos:
          # assert not anno["region_attributes"]
          anno = dict_anno["shape_attributes"]
          px = anno["all_points_x"]
          py = anno["all_points_y"]
          poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
          poly = [p for x in poly for p in x]

          # get type from region_attributes to set different category_id
          attr1 = dict_anno["region_attributes"]
          type1 = attr1["type"]

          if type1 == "abrasive wear":
              cat_id = 0
          elif type1 == "adhesive wear":
              cat_id = 1
          elif type1 == "fatigue wear":
              cat_id = 2
          else:
              cat_id = 0

          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": cat_id,
          }
          objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
  return dataset_dicts

for d in ["train", "val"]:
  DatasetCatalog.register("wear_" + d, lambda d=d: get_wear_dicts(os.path.join("/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108", d)))
  MetadataCatalog.get("wear_" + d).set(thing_classes=["abrasive wear","adhesive wear","fatigue wear"])
wear_metadata = MetadataCatalog.get("wear_train")

### 2.1B Register with this one if you label with labelme
### 2.1B 注册图像数据集(Labelme标注的用这个)

In [ ]:
def get_wear_dicts(directory):
  classes = ['abrasive wear', 'adhesive wear', 'fatigue wear']
  dataset_dicts = []
  for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
      json_file = os.path.join(directory, filename)
      with open(json_file) as f:
          img_anns = json.load(f)

      record = {}
      
      filename = os.path.join(directory, img_anns["imagePath"])
      
      record["file_name"] = filename
      record["height"] = 1088
      record["width"] = 1024
    
      annos = img_anns["shapes"]
      objs = []
      for anno in annos:
          px = [a[0] for a in anno['points']]
          py = [a[1] for a in anno['points']]
          poly = [(x, y) for x, y in zip(px, py)]
          poly = [p for x in poly for p in x]

          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": classes.index(anno['label']),
              "iscrowd": 0
          }
          objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
  return dataset_dicts

for d in ["train", "val"]:
  DatasetCatalog.register("wear_" + d, lambda d=d: get_wear_dicts("/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/" + d))
  MetadataCatalog.get("wear_" + d).set(thing_classes=['abrasive wear', 'adhesive wear', 'fatigue wear'])
wear_metadata = MetadataCatalog.get("wear_train")

### 2.2 Visualize the annotations of randomly selected samples in the training set
### 2.2 随机预览训练集图像分割方式

In [ ]:
dataset_dicts = get_wear_dicts("/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/train")
for d in random.sample(dataset_dicts, 1):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=wear_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

### 2.3 Save labelled results of val dataset in folder
### 2.3 保存验证集图片的标注结果输出到文件夹

In [ ]:
def get_dir_true_info(directory):
  os.mkdir(directory+'_true')
  csv_file = open(directory+'_true/info.csv', 'w')
  csv_file.close()
  text_title = 'file_name,height,width,pixel,instance_num,abrasive_area,adhesive_area,fatigue_area'
  with open(directory+'_true/info.csv','a+',encoding='utf-8') as f:
    f.write(text_title+'\n')
    f.close()
  dataset_dicts = get_wear_dicts(directory)
  for d in dataset_dicts:
    file = d["file_name"]
    file_name = file.split("/")[-1]
    true_name = directory+'_true/'+file_name
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=wear_metadata, scale=1.0)
    out = visualizer.draw_dataset_dict(d)
    # cv2_imshow(out.get_image()[:, :, ::-1])
    cv2.imwrite(true_name, out.get_image()[:, :, ::-1])
  dir_info = []
  for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
    json_file = os.path.join(directory, filename)
    with open(json_file) as f:
      img_json = json.load(f)
      f.close()
    annos = img_json["shapes"]
    annos_num = len(annos)
    img_name = img_json["imagePath"]
    img_height = img_json["imageHeight"]
    img_width = img_json["imageWidth"]
    img_pixel = img_height * img_width
    abrasive_mask = np.zeros((img_height, img_width))
    adhesive_mask = np.zeros((img_height, img_width))
    fatigue_mask = np.zeros((img_height, img_width))   
    for i, x in enumerate(annos):
      polygon_mask = np.zeros((img_height, img_width))
      points = x["points"]
      pts = np.array(points, dtype=np.int32)
      cv2.fillConvexPoly(polygon_mask, pts, 1)
      if x['label'] == 'abrasive wear':
        abrasive_mask = np.array(abrasive_mask) + np.array(polygon_mask)
        abrasive_mask[abrasive_mask > 0] = 1
      elif x['label'] == 'adhesive wear':
        adhesive_mask = np.array(adhesive_mask) + np.array(polygon_mask)
        adhesive_mask[adhesive_mask > 0] = 1
      elif x['label'] == 'fatigue wear':
        fatigue_mask = np.array(fatigue_mask) + np.array(polygon_mask)
        fatigue_mask[fatigue_mask > 0] = 1
    abrasive_area = np.sum(abrasive_mask != 0)
    adhesive_area = np.sum(adhesive_mask != 0)
    fatigue_area = np.sum(fatigue_mask != 0)
    image_info = {"img_name":img_name, "abrasive_mask":abrasive_mask,"adhesive_mask":adhesive_mask,"fatigue_mask":fatigue_mask}
    dir_info.append(image_info)
    true_text = str(img_name)+','+str(img_height)+','+str(img_width)+','+str(img_pixel)+','+str(annos_num)+','+str(abrasive_area)+','+str(adhesive_area)+','+str(fatigue_area)
    with open((directory+'_true/info.csv'),'a+',encoding='utf-8') as f:
      f.write(true_text+'\n')
      f.close()
  return dir_info

train_true_info = get_dir_true_info('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/train')
val_true_info = get_dir_true_info('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/val')

## 3. Mask R-CNN → Instance Segmentation
## 3. Mask R-CNN实例分割训练模型

### 3.1 Train! (Or skip to use the pre-trained model)
### 3.1 训练! (或跳过这一步直接使用已训练的模型）
> 

> 

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("wear_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml") 
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025   # pick a good LR
cfg.SOLVER.MAX_ITER = 10000   # you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # default: 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3   

cfg.OUTPUT_DIR = '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/output'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
print('train done.')

### 3.2 Check training curves
### 3.2 查看训练曲线
> Check training curves of pre-trained model by default

> 默认查看已训练好了的模型曲线

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
# %tensorboard --logdir '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/output'
%tensorboard --logdir '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/WC-Cu_20210108_output'

### 3.3 Inference! (The pre-trained model is applied by default)
### 3.3 预测! (默认使用已训练过的模型预测)
> 

> 

In [ ]:
def get_dir_pred_info(directory):
  os.mkdir(directory+'_pred')
  csv_file = open(directory+'_pred/info.csv', 'w')
  text_title = 'file_name,height,width,pixel,instance_num,abrasive_area,adhesive_area,fatigue_area'
  with open(directory+'_pred/info.csv','a+',encoding='utf-8') as f:
    f.write(text_title+'\n')
    f.close()
  dir_info = []
  files = os.listdir(directory)
  files.sort()
  for file_name in files:
    # filter jpg and tiff files
    if file_name[-4:] == '.jpg' or file_name[-5:] == '.tiff':
      image_path = os.path.join(directory, file_name)
      im = cv2.imread(image_path)
      outputs = predictor(im)          
      predictions = outputs["instances"].to("cpu")
      predictions_classes = np.asarray(predictions.pred_classes)
      predictions_scores = np.asarray(predictions.scores)
      predictions_masks = np.asarray(predictions.pred_masks)
      predictions_areas = []
      predictions_instance_num = predictions_masks.shape[0]
      predictions_height = predictions_masks.shape[1]
      predictions_width = predictions_masks.shape[2]
      predictions_pixel = predictions_height * predictions_width          
      blank_mask = np.zeros((predictions_height, predictions_width), dtype=bool)
      predictions_abrasive_mask = blank_mask
      predictions_adhesive_mask = blank_mask
      predictions_fatigue_mask = blank_mask
      for i in range(0, predictions_instance_num):
        instance_num = i
        instance_class = predictions_classes[i]
        instance_score = predictions_scores[i]
        instance_mask = predictions_masks[i]
        instance_area = np.sum(instance_mask != 0)
        predictions_areas.append(instance_area)
        if predictions_classes[i] == 0:
          predictions_abrasive_mask = np.array(predictions_abrasive_mask) + np.array(instance_mask)
          predictions_abrasive_mask[predictions_abrasive_mask > 0] = 1
        elif predictions_classes[i] == 1:
          predictions_adhesive_mask = np.array(predictions_adhesive_mask) + np.array(instance_mask)
          predictions_adhesive_mask[predictions_adhesive_mask > 0] = 1
        elif predictions_classes[i] == 2:
          predictions_fatigue_mask = np.array(predictions_fatigue_mask) + np.array(instance_mask)
          predictions_fatigue_mask[predictions_fatigue_mask > 0] = 1       
      predictions_abrasive_area = np.sum(predictions_abrasive_mask != 0)
      predictions_adhesive_area = np.sum(predictions_adhesive_mask != 0)
      predictions_fatigue_area = np.sum(predictions_fatigue_mask != 0)
      image_info = {"img_name":file_name, "abrasive_mask":predictions_abrasive_mask,"adhesive_mask":predictions_adhesive_mask,"fatigue_mask":predictions_fatigue_mask}
      dir_info.append(image_info)
      predictions_text = str(file_name)+','+str(predictions_height)+','+str(predictions_width)+','+str(predictions_pixel)+','+str(predictions_instance_num)+','+str(predictions_abrasive_area)+','+str(predictions_adhesive_area)+','+str(predictions_fatigue_area)
      with open(directory+'_pred/info.csv','a+',encoding='utf-8') as f:
        f.write(predictions_text+'\n')
        f.close()
      v = Visualizer(im[:,:,::-1], metadata=wear_metadata, scale=1.0, instance_mode=ColorMode.IMAGE_BW)
      out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
      image_obj = out.get_image()[:, :, ::-1]
      # cv2_imshow(image_obj)
      cv2.imwrite(os.path.join((directory+'_pred'), file_name), image_obj)
  return dir_info

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.MODEL.WEIGHTS = os.path.join('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/WC-Cu_20210108_output', "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

train_pred_info = get_dir_pred_info('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/train')
val_pred_info = get_dir_pred_info('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/val')
test_pred_info = get_dir_pred_info('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/test')

### 3.4 Evaluate inferencing results of all wear modes
### 3.4 评价各磨损类型预测效果

In [ ]:
!touch '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/train_count.csv'
!touch '/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/val_count.csv'
text_title = 'img_name,abrasive_mask_iou,adhesive_mask_iou,fatigue_mask_iou,abrasive_area_loss,adhesive_area_loss,fatigue_area_loss'
with open('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/train_count.csv','a+',encoding='utf-8') as f:
  f.write(text_title+'\n')
  f.close()
with open('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/val_count.csv','a+',encoding='utf-8') as f:
  f.write(text_title+'\n')
  f.close()

def mask_iou(mask1, mask2):
  area1 = mask1.sum()
  area2 = mask2.sum()
  if area1 + area2 == 0:
    mask_iou = 1
  else:
    inter = ((mask1+mask2)==2).sum()
    mask_iou = inter / (area1+area2-inter)
  return mask_iou

def area_loss(mask1, mask2):
  area1 = mask1.sum()
  area2 = mask2.sum()
  if area1 + area2 == 0:
    area_loss = 0
  else:
    area_loss = abs(area1 - area2)/(area1 + area2)
  return area_loss

for i, x in enumerate(train_true_info):
  img_name = x["img_name"]
  true_abrasive_mask = x["abrasive_mask"]
  true_adhesive_mask = x["adhesive_mask"]
  true_fatigue_mask = x["fatigue_mask"]
  pred_abrasive_mask = train_pred_info[i]["abrasive_mask"]
  pred_adhesive_mask = train_pred_info[i]["adhesive_mask"]
  pred_fatigue_mask = train_pred_info[i]["fatigue_mask"]
  train_abrasive_iou = mask_iou(true_abrasive_mask, pred_abrasive_mask)
  train_adhesive_iou = mask_iou(true_adhesive_mask, pred_adhesive_mask)
  train_fatigue_iou = mask_iou(true_fatigue_mask, pred_fatigue_mask)
  train_abrasive_loss = area_loss(true_abrasive_mask, pred_abrasive_mask)
  train_adhesive_loss = area_loss(true_adhesive_mask, pred_adhesive_mask)
  train_fatigue_loss = area_loss(true_fatigue_mask, pred_fatigue_mask)
  count_text = str(img_name)+','+str(train_abrasive_iou)+','+str(train_adhesive_iou)+','+str(train_fatigue_iou)+','+str(train_abrasive_loss)+','+str(train_adhesive_loss)+','+str(train_fatigue_loss)
  with open('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/train_count.csv','a+',encoding='utf-8') as f:
    f.write(count_text+'\n')
    f.close()

for i, x in enumerate(val_true_info):
  img_name = x["img_name"]
  true_abrasive_mask = x["abrasive_mask"]
  true_adhesive_mask = x["adhesive_mask"]
  true_fatigue_mask = x["fatigue_mask"]
  pred_abrasive_mask = val_pred_info[i]["abrasive_mask"]
  pred_adhesive_mask = val_pred_info[i]["adhesive_mask"]
  pred_fatigue_mask = val_pred_info[i]["fatigue_mask"]
  val_abrasive_iou = mask_iou(true_abrasive_mask, pred_abrasive_mask)
  val_adhesive_iou = mask_iou(true_adhesive_mask, pred_adhesive_mask)
  val_fatigue_iou = mask_iou(true_fatigue_mask, pred_fatigue_mask)
  val_abrasive_loss = area_loss(true_abrasive_mask, pred_abrasive_mask)
  val_adhesive_loss = area_loss(true_adhesive_mask, pred_adhesive_mask)
  val_fatigue_loss = area_loss(true_fatigue_mask, pred_fatigue_mask)
  count_text = str(img_name)+','+str(val_abrasive_iou)+','+str(val_adhesive_iou)+','+str(val_fatigue_iou)+','+str(val_abrasive_loss)+','+str(val_adhesive_loss)+','+str(val_fatigue_loss)
  with open('/content/drive/MyDrive/IDB_Wear/WC-Cu_20210108/val_count.csv','a+',encoding='utf-8') as f:
    f.write(count_text+'\n')
    f.close()

## 4. Other models
## 4. 其他模型

### 4.1 关键点检测

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

### 4.2 全景分割

In [ ]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])

### 4.3 视频全景分割

In [ ]:
# This is the video we're going to process
from IPython.display import YouTubeVideo, display
video = YouTubeVideo("ll8TgCZ0plk", width=500)
display(video)

In [ ]:
# Install dependencies, download the video, and crop 5 seconds for processing
!pip install youtube-dl
!pip uninstall -y opencv-python-headless opencv-contrib-python
!apt install python3-opencv  # the one pre-installed have some issues
!youtube-dl https://www.youtube.com/watch?v=ll8TgCZ0plk -f 22 -o video.mp4
!ffmpeg -i video.mp4 -t 00:00:06 -c:v copy video-clip.mp4

In [ ]:
# Run frame-by-frame inference demo on this video (takes 3-4 minutes) with the "demo.py" tool we provided in the repo.
!git clone https://github.com/facebookresearch/detectron2
!python detectron2/demo/demo.py --config-file detectron2/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml --video-input video-clip.mp4 --confidence-threshold 0.6 --output video-output.mkv \
  --opts MODEL.WEIGHTS detectron2://COCO-PanopticSegmentation/panoptic_fpn_R_101_3x/139514519/model_final_cafdb1.pkl

In [ ]:
# Download the results
from google.colab import files
files.download('video-output.mkv')